In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('a.csv')

In [3]:
df['Сall forwarding']=df['Сall forwarding'].fillna(0)

In [4]:
list(df)

['sender',
 'Notes',
 'Nickname',
 'Priority',
 'Рosition',
 'Сompany',
 'Сall forwarding',
 'E-Mail ',
 'Сontent']

In [5]:
df=df.rename(columns={'Сall forwarding':'call_forw','E-Mail ':'to'})

## Создание словаря и его бинаризация

In [6]:
buf=df.loc[df['call_forw'] == '*']
buf=buf.drop(columns='to')
buf['to']=buf['Notes']



In [114]:
df=df[df['call_forw'] != '*'].append(buf,sort=True,ignore_index=True)

In [115]:
df.head()

Nickname Notes  Priority call_forw                          sender  \
0         k_o_1   NaN       NaN         0       Закиров Руслан Ильнурович   
1        nov_gr   NaN       NaN         0       Сабитова Регина Шамилевна   
2      liona499   NaN       NaN         0        Гимадеева Алсу Айратовна   
3  chugaev-mark   NaN       NaN         0   Зайцева Александра Евгеньевна   
4  Krajushkin.M   NaN       NaN         0  Ширяев Александр Владиславович   

                         to  Рosition  Сompany  \
0           k_o_1@yandex.ru       NaN      NaN   
1            nov_gr@mail.ru       NaN      NaN   
2       liona499@rambler.ru       NaN      NaN   
3   chugaev-mark@rambler.ru       NaN      NaN   
4  krajushkin.m@sch2009.net       NaN      NaN   

                                             Сontent  
0                       Поздравление с днём рождения  
1                                             Опросы  
2  программное обеспечение, оборудование или услу...  
3  программное обеспечение, оборудование или услу...  
4                          Персональные рекомендации

In [116]:
df=df.drop(columns=['Notes','call_forw','Nickname','Рosition','Priority','Сompany'])

In [117]:
df['Сontent'].nunique()

13

In [118]:
df_bag_of_words=pd.get_dummies(df['Сontent'])

In [121]:
df.drop(columns='Сontent').head()
df=df.drop(columns='Сontent')

In [125]:
print(df_bag_of_words.shape)
print(df.shape)


(380, 13)
(380, 2)


In [127]:
df=pd.concat((df,df_bag_of_words),axis=1)


In [128]:
df['to'].nunique()

359

## Разделение записей с несколькими пользователями

In [78]:
def normalize(s):
    try:
        zn=str()
        s=s.lower()
        for i in range(len(s)):
            if(not s[i].isalpha() ):
                s=s[:i]+' '+s[i+1:]
        for word in s.split(' '):
            if(len(word)>=4):
                zn=zn+word+' '
        return zn[:-1]
    except AttributeError:
        return None

In [79]:
df['Сontent']=df['Сontent'].apply(normalize)

In [81]:
tr=df.head()

In [82]:
tr=tr.rename(columns={'Сontent':'content'})

In [91]:
tr=tr.reset_index()

In [101]:
tr=tr.rename(columns={'index':'id'})

In [102]:
new_df=pd.DataFrame(data=tr.content.str.split(' ').tolist(),index=tr.id).stack()
new_df

id   
0   0      программное
    1      обеспечение
    2     оборудование
    3           услуги
    4    автоматизации
    5          бизнеса
1   0     поздравление
    1             днём
    2         рождения
2   0           опросы
3   0     персональные
    1     рекомендации
4   0      программное
    1      обеспечение
    2     оборудование
    3           услуги
    4    автоматизации
    5          бизнеса
dtype: object

In [104]:
new_df=new_df.reset_index([0,'id'])

In [107]:
pd.merge(new_df,tr,on='id')

id              0                         sender                  Notes  \
0    0    программное  Базаева Анастасия Анатольевна  levshin.m@sch2009.net   
1    0    обеспечение  Базаева Анастасия Анатольевна  levshin.m@sch2009.net   
2    0   оборудование  Базаева Анастасия Анатольевна  levshin.m@sch2009.net   
3    0         услуги  Базаева Анастасия Анатольевна  levshin.m@sch2009.net   
4    0  автоматизации  Базаева Анастасия Анатольевна  levshin.m@sch2009.net   
5    0        бизнеса  Базаева Анастасия Анатольевна  levshin.m@sch2009.net   
6    1   поздравление      Закиров Руслан Ильнурович                    NaN   
7    1           днём      Закиров Руслан Ильнурович                    NaN   
8    1       рождения      Закиров Руслан Ильнурович                    NaN   
9    2         опросы      Сабитова Регина Шамилевна                    NaN   
10   3   персональные     Садреева Эльвира Илнуровна        marina@kerby.ru   
11   3   рекомендации     Садреева Эльвира Илнуровна        marina@kerby.ru   
12   4    программное       Гимадеева Алсу Айратовна                    NaN   
13   4    обеспечение       Гимадеева Алсу Айратовна                    NaN   
14   4   оборудование       Гимадеева Алсу Айратовна                    NaN   
15   4         услуги       Гимадеева Алсу Айратовна                    NaN   
16   4  автоматизации       Гимадеева Алсу Айратовна                    NaN   
17   4        бизнеса       Гимадеева Алсу Айратовна                    NaN   

        Nickname  Priority  Рosition  Сompany call_forw  \
0     29.10.1924       NaN       NaN      NaN         *   
1     29.10.1924       NaN       NaN      NaN         *   
2     29.10.1924       NaN       NaN      NaN         *   
3     29.10.1924       NaN       NaN      NaN         *   
4     29.10.1924       NaN       NaN      NaN         *   
5     29.10.1924       NaN       NaN      NaN         *   
6          k_o_1       NaN       NaN      NaN         0   
7          k_o_1       NaN       NaN      NaN         0   
8          k_o_1       NaN       NaN      NaN         0   
9         nov_gr       NaN       NaN      NaN         0   
10  Артём Крицын       NaN       NaN      NaN         *   
11  Артём Крицын       NaN       NaN      NaN         *   
12      liona499       NaN       NaN      NaN         0   
13      liona499       NaN       NaN      NaN         0   
14      liona499       NaN       NaN      NaN         0   
15      liona499       NaN       NaN      NaN         0   
16      liona499       NaN       NaN      NaN         0   
17      liona499       NaN       NaN      NaN         0   

                        to                                            content  
0       29.10.1924@mail.ru  программное обеспечение оборудование услуги ав...  
1       29.10.1924@mail.ru  программное обеспечение оборудование услуги ав...  
2       29.10.1924@mail.ru  программное обеспечение оборудование услуги ав...  
3       29.10.1924@mail.ru  программное обеспечение оборудование услуги ав...  
4       29.10.1924@mail.ru  программное обеспечение оборудование услуги ав...  
5       29.10.1924@mail.ru  программное обеспечение оборудование услуги ав...  
6          k_o_1@yandex.ru                         поздравление днём рождения  
7          k_o_1@yandex.ru                         поздравление днём рождения  
8          k_o_1@yandex.ru                         поздравление днём рождения  
9           nov_gr@mail.ru                                             опросы  
10  kritsynartem@gmail.com                          персональные рекомендации  
11  kritsynartem@gmail.com                          персональные рекомендации  
12     liona499@rambler.ru  программное обеспечение оборудование услуги ав...  
13     liona499@rambler.ru  программное обеспечение оборудование услуги ав...  
14     liona499@rambler.ru  программное обеспечение оборудование услуги ав...  
15     liona499@rambler.ru  программное обеспечение оборудование услуги ав...  
16     liona499@ramb

In [106]:
tr

id                         sender                  Notes      Nickname  \
0   0  Базаева Анастасия Анатольевна  levshin.m@sch2009.net    29.10.1924   
1   1      Закиров Руслан Ильнурович                    NaN         k_o_1   
2   2      Сабитова Регина Шамилевна                    NaN        nov_gr   
3   3     Садреева Эльвира Илнуровна        marina@kerby.ru  Артём Крицын   
4   4       Гимадеева Алсу Айратовна                    NaN      liona499   

   Priority  Рosition  Сompany call_forw                      to  \
0       NaN       NaN      NaN         *      29.10.1924@mail.ru   
1       NaN       NaN      NaN         0         k_o_1@yandex.ru   
2       NaN       NaN      NaN         0          nov_gr@mail.ru   
3       NaN       NaN      NaN         *  kritsynartem@gmail.com   
4       NaN       NaN      NaN         0     liona499@rambler.ru   

                                             content  
0  программное обеспечение оборудование услуги ав...  
1                         поздравление днём рождения  
2                                             опросы  
3                          персональные рекомендации  
4  программное обеспечение оборудование услуги ав...

In [110]:
example_df=pd.DataFrame(data={'to':['user1','user2','user3','user1\nuser3','user2\nuser3'],'note':[134,8763,7222,6111,55]})

In [153]:
new_from_example_df=pd.DataFrame(data=example_df.to.str.split('\n').tolist()).stack()
new_from_example_df.to_frame()
new_from_example_df=new_from_example_df.reset_index().drop(columns=['level_1'])

In [139]:
example_df=example_df.reset_index().rename(columns={'index':'id'})

In [146]:
new_from_example_df

0
0  user1
1  user2
2  user3
3  user1
4  user3
5  user2
6  user3

In [156]:
pd.merge(example_df.drop(columns='id'),new_from_example_df,left_index=True,right_on='level_0').drop(columns=['to','level_0']).\
    rename(columns={0:'to'})

note     to
0   134  user1
1  8763  user2
2  7222  user3
3  6111  user1
4  6111  user3
5    55  user2
6    55  user3